In [10]:
from utils.sampling import *
from utils.config import cfg_from_yaml_file
from tools import builder
from datasets.data_transforms import ResamplePoints, LidarSimulation
from datasets import build_dataset_from_cfg
import open3d as o3d
import numpy as np
import glob
from utils.vis_utils import convert_to_o3dpcd

In [11]:
from pathlib import Path
import pickle

data_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/val')
modelid_path = data_dir / 'model_ids.txt'

with open(modelid_path, 'r') as f:
    model_ids = [line.split('\n')[0] for line in f.readlines()]

partial_points_path = str(data_dir / 'partial' / '%s' / '%03d.pcd')
surface_points_path = str(data_dir / 'surface' / '%s' / '%03d.pcd')
label_path = str(data_dir / 'label.pkl')

with open(label_path, 'rb') as f:
    label = pickle.load(f)

In [14]:
from tqdm import tqdm 

lidarsim = LidarSimulation({})

dataset_size = 2000
data_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test')
partial_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test/partial')
surface_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test/surface')
num_exported = 0

new_labels = {}
for i in tqdm(range(len(model_ids)), total=len(model_ids)):
    
    m_id = model_ids[i]  
    new_labels[m_id] = {}
    
    for v_id in range(20):             
        partial_path = partial_points_path % (m_id, v_id)
        surface_path = surface_points_path % (m_id, v_id)

        pcd = o3d.io.read_point_cloud(partial_path)
        pc_np = np.asarray(pcd.points)
        pc_np_sim = lidarsim(pc_np)
        partial_pc_path = partial_dir / m_id
        partial_pc_path.mkdir(exist_ok=True, parents=True)        
        o3d.io.write_point_cloud(str(partial_pc_path / f'{v_id:03d}.pcd'), convert_to_o3dpcd(pc_np_sim))
        
        surface_pcd = o3d.io.read_point_cloud(surface_path)
        surface_pc_path = surface_dir / m_id
        surface_pc_path.mkdir(exist_ok=True, parents=True) 
        o3d.io.write_point_cloud(str(surface_pc_path / f'{v_id:03d}.pcd'), surface_pcd)
        
        new_labels[m_id][v_id] = label[m_id][v_id]
        new_labels[m_id][v_id]['num_pts'] = pc_np_sim.shape[0]        

with open(str(data_dir / 'label.pkl'), 'wb') as f:
    pickle.dump(new_labels, f)

100%|█████████████████████████████████████████████| 340/340 [01:22<00:00,  4.10it/s]


In [18]:
6*len(model_ids)

2040

In [9]:
from pathlib import Path

pth = '/VCN/data/shapenet/VC/vc-surface_nviews-20/test/surface/1ed012d3232de232473f10e6caaeca56/000.pcd'
print(Path(pth).exists())

pcd = o3d.io.read_point_cloud(pth)

False
[Open3D WARNING] Read PCD failed: unable to open file: /VCN/data/shapenet/VC/vc-surface_nviews-20/test/surface/1ed012d3232de232473f10e6caaeca56/000.pcd


In [60]:
label_dir = '/VCN/data/shapenet/VC/test_2000/kitti/test/label'
pc_dir = '/VCN/data/shapenet/VC/test_2000/kitti/test/partial'
label_pths = glob.glob(label_dir + '/*.pkl')

combined = {}
for pth in label_pths:
    with open(pth, 'rb') as f:
        label = pickle.load(f)
    
    pc_id = label['pc_id']
    pc = o3d.io.read_point_cloud(pc_dir + f'/{pc_id}.pcd')
            
    combined[pc_id] = label
    combined['dataset'] = 'kitti'
    combined[pc_id]['num_pts'] = len(pc.points)    
    
new_lab_path = '/VCN/data/shapenet/VC/test_2000/kitti/test/label.pkl'
with open(new_lab_path, 'wb') as f:
    pickle.dump(combined, f)

In [61]:
pth = '/VCN/data/shapenet/VC/test_2000/kitti/test/label.pkl'
with open(pth, 'rb') as f:
    plabel = pickle.load(f)

In [63]:
plabel

{'frame-125_car-004': {'bbox_pts': array([[39.16177604, 11.38062185, -1.53117708],
         [39.91692298, 14.58278567, -1.53117708],
         [37.60449273, 11.74786657, -1.53117708],
         [39.16177604, 11.38062185,  0.06882292],
         [38.35963967, 14.95003038,  0.06882292],
         [37.60449273, 11.74786657,  0.06882292],
         [39.91692298, 14.58278567,  0.06882292],
         [38.35963967, 14.95003038, -1.53117708]]),
  'gtbox': array([38.76070786, 13.16532612, -0.93117708,  3.29      ,  1.6       ,
          1.6       ,  1.33920367]),
  'pc_id': 'frame-125_car-004',
  'num_pts': 97},
 'dataset': 'kitti',
 'frame-50_car-000': {'bbox_pts': array([[12.8836347 , -3.31577827, -1.23447887],
         [17.22283447, -3.23243945, -1.23447887],
         [12.85367882, -1.75606591, -1.23447887],
         [12.8836347 , -3.31577827,  0.25552113],
         [17.19287859, -1.67272709,  0.25552113],
         [12.85367882, -1.75606591,  0.25552113],
         [17.22283447, -3.23243945,  0.255